# Pre-processing 

### Import libraries and read the new cleaned csv file

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from datasist.structdata import detect_outliers
import plotly.figure_factory as ff
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from category_encoders import BinaryEncoder
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler

In [2]:
df2=pd.read_csv(r"Preprocessing and bi-variant analysis.csv")

In [3]:
df2

,address,price_in_dollars,number_of_bedrooms,number_of_baths,area in sq.meter,city,state
0,1866 20th Ave,294900,1.0,1.0,89.557785,Arkdale,Wisconsin
1,1639 Pine St,89500,2.0,1.0,85.841695,Arkdale,Wisconsin
2,1644 20th Ave,475000,3.0,2.0,202.898551,Arkdale,Wisconsin
3,2054 Cumberland Ave,179000,4.0,1.0,162.021553,Arkdale,Wisconsin
4,1841 20th Ct,359000,3.0,2.0,156.911929,Arkdale,Wisconsin
...,...,...,...,...,...,...,...
1439,"Albany, 12211",735400,2.0,3.0,186.826459,Albany,New york
1440,"Albany, 12211",795400,3.0,3.0,206.800446,Albany,New york
1441,"Albany, 12211",705400,2.0,2.0,183.574879,Albany,New york
1442,"Albany, 12211",815400,4.0,3.0,267.837235,Albany,New york


In [4]:
df2.isnull().sum()  ## There are no null values for categorical data

address                 0
price_in_dollars        0
number_of_bedrooms      6
number_of_baths       107
area in sq.meter       31
city                    0
state                   0
dtype: int64

In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1444 entries, 0 to 1443
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   address             1444 non-null   object 
 1   price_in_dollars    1444 non-null   int64  
 2   number_of_bedrooms  1438 non-null   float64
 3   number_of_baths     1337 non-null   float64
 4   area in sq.meter    1413 non-null   float64
 5   city                1444 non-null   object 
 6   state               1444 non-null   object 
dtypes: float64(3), int64(1), object(3)
memory usage: 79.1+ KB


## First step to check outliers for numerical columns

### a-Price column

#### Price column can have outliers as the price depends on several feature more than presented in data for example does it have a garden, terrace, roof , renovated and wether the building is old or new so it is not logic to judge the outliers of price.

### b-Number of  bathrooms columns

#### Number of bedrooms and bathrooms outliers can be identified from size roughly

In [6]:
df2['area in sq.meter'].describe()

count    1413.000000
mean      222.028265
std       160.288829
min        31.400966
25%       130.063174
50%       190.914158
75%       265.793385
max      2632.199926
Name: area in sq.meter, dtype: float64

In [7]:
outliers_bath= df2[df2['number_of_baths']>7]  ##value 7 is obtained from high whisker point in visualization in data cleaning notebook

In [8]:
outliers_bath

,address,price_in_dollars,number_of_bedrooms,number_of_baths,area in sq.meter,city,state
268,1517 Lone Oak Cir,4375000,5.0,9.0,637.030844,Nashville,Tennessee
315,4805 Churchwood Dr,4950000,5.0,8.0,809.457451,Nashville,Tennessee
316,213 3rd Ave N,16680000,11.0,11.0,1292.177629,Nashville,Tennessee
344,512 Fairfax Ave,3600000,6.0,8.0,665.830546,Nashville,Tennessee
359,6 Castlewood Ct,3399000,6.0,8.0,718.970643,Nashville,Tennessee
435,3013 Medial Ave,3690000,6.0,8.0,579.338536,Nashville,Tennessee
639,1300 Harding Pl,4199923,5.0,8.0,640.932739,Nashville,Tennessee
660,102 Clydelan Ct,3395000,5.0,8.0,498.792271,Nashville,Tennessee
776,5028 Franklin Pike,5800000,7.0,12.0,1094.202899,Nashville,Tennessee
831,1108 Overton Lea Rd,5500000,4.0,8.0,675.585284,Nashville,Tennessee


### Assuming that area of 150 square m wether a house or appartment can have at maximum 3 bedrooms and 2 bathrooms and area of 300 square m can have 6 bedrooms and 4 bathrooms and area of 450 square m can have at maximum 9 bedrooms and 6 bathrooms and so on.

### For number of bathrooms there is no outliers and number of bathrooms matches the assumption and also the number of bedrooms matches the assumption.

### b-Number_of_bedrooms columns


In [9]:
outliers_bedrooms=df2[df2['number_of_bedrooms']>5] ##value 5 is obtained from high whisker point in visualization in data cleaning notebook

In [10]:
outliers_bedrooms

,address,price_in_dollars,number_of_bedrooms,number_of_baths,area in sq.meter,city,state
93,1315 Lone Oak Cir,3890000,6.0,6.0,597.640282,Nashville,Tennessee
104,122 38th Ave N,2199000,6.0,6.0,451.783724,Nashville,Tennessee
120,1025 Carolyn Ave E,1150000,6.0,3.0,373.374210,Nashville,Tennessee
223,860 Clematis Drive,629000,6.0,4.0,284.280936,Nashville,Tennessee
237,1606 16th Ave S,2199800,7.0,5.0,279.264214,Nashville,Tennessee
...,...,...,...,...,...,...,...
1424,28 Robin Street,575000,9.0,NaN,560.386473,Albany,New york
1425,183 N Main Avenue,799000,10.0,NaN,738.758826,Albany,New york
1426,555 1st Street,139000,6.0,NaN,NaN,Albany,New york
1428,284 Morton Avenue,160000,6.0,NaN,227.053140,Albany,New york


### Since the number of rows is numerous, therefore outliers can not be detected by eye as in outliers of number of bathrooms and this concept will be applied by a function

In [11]:
idx4=outliers_bedrooms.index ##index of outliers of column number_of_bathrooms

In [12]:
idx4

Int64Index([  93,  104,  120,  223,  237,  316,  344,  359,  435,  496,
            ...
            1407, 1408, 1410, 1419, 1423, 1424, 1425, 1426, 1428, 1435],
           dtype='int64', length=106)

In [13]:
outliers_bedrooms['area in sq.meter'].describe()

count      96.000000
mean      377.467716
std       246.628758
min       115.942029
25%       217.391304
50%       293.199554
75%       459.982349
max      1427.722036
Name: area in sq.meter, dtype: float64

In [14]:
mat={1500:[30,20],1350:[27,18],1200:[24,16],1050:[21,14],900:[18,12],750:[15,10],600:[12,8],450:[9,6],300:[6,4],150:[3,2],75:[2,2]}

In [15]:
"""
This for loop is used number_of_bedrooms outliers column first it determines the range for the value of area as shown in
dictionary above 'mat', then when there is a nan value in either area or bathroom it remains the same without change but if 
all values are present it compares the number of bedrooms and bathrooms with the supposed to be in 'mat', if they are less then
they remain as the same but if they are greater, then we modify them to be the same as 'mat'
"""
for i in idx4:
    if (pd.isnull(outliers_bedrooms.loc[i]['number_of_baths'])) or (pd.isnull(outliers_bedrooms.loc[i]['area in sq.meter'])):
        continue
    else:
        for x in mat:
            if outliers_bedrooms.loc[i]['area in sq.meter'] >= x:
                if outliers_bedrooms.loc[i]['number_of_bedrooms'] >= mat[x][0]:
                    df2.loc[i]['number_of_bedrooms'] = mat[x][0]
                if outliers_bedrooms.loc[i]['number_of_baths'] >= mat[x][1]:
                    df2.loc[i]['number_of_baths'] = mat[x][1]
        else:
                continue
                
                    
                                                                    
                        
                                                                    

In [16]:
df2.iloc[idx4]

,address,price_in_dollars,number_of_bedrooms,number_of_baths,area in sq.meter,city,state
93,1315 Lone Oak Cir,3890000,6.0,6.0,597.640282,Nashville,Tennessee
104,122 38th Ave N,2199000,6.0,6.0,451.783724,Nashville,Tennessee
120,1025 Carolyn Ave E,1150000,6.0,3.0,373.374210,Nashville,Tennessee
223,860 Clematis Drive,629000,6.0,4.0,284.280936,Nashville,Tennessee
237,1606 16th Ave S,2199800,7.0,5.0,279.264214,Nashville,Tennessee
...,...,...,...,...,...,...,...
1424,28 Robin Street,575000,9.0,NaN,560.386473,Albany,New york
1425,183 N Main Avenue,799000,10.0,NaN,738.758826,Albany,New york
1426,555 1st Street,139000,6.0,NaN,NaN,Albany,New york
1428,284 Morton Avenue,160000,6.0,NaN,227.053140,Albany,New york


### c-area in sq.meter column


In [17]:
df2

,address,price_in_dollars,number_of_bedrooms,number_of_baths,area in sq.meter,city,state
0,1866 20th Ave,294900,1.0,1.0,89.557785,Arkdale,Wisconsin
1,1639 Pine St,89500,2.0,1.0,85.841695,Arkdale,Wisconsin
2,1644 20th Ave,475000,3.0,2.0,202.898551,Arkdale,Wisconsin
3,2054 Cumberland Ave,179000,4.0,1.0,162.021553,Arkdale,Wisconsin
4,1841 20th Ct,359000,3.0,2.0,156.911929,Arkdale,Wisconsin
...,...,...,...,...,...,...,...
1439,"Albany, 12211",735400,2.0,3.0,186.826459,Albany,New york
1440,"Albany, 12211",795400,3.0,3.0,206.800446,Albany,New york
1441,"Albany, 12211",705400,2.0,2.0,183.574879,Albany,New york
1442,"Albany, 12211",815400,4.0,3.0,267.837235,Albany,New york


In [18]:
df2['area in sq.meter'].describe()

count    1413.000000
mean      222.028265
std       160.288829
min        31.400966
25%       130.063174
50%       190.914158
75%       265.793385
max      2632.199926
Name: area in sq.meter, dtype: float64

In [19]:
outliers_area =df2[(df2['area in sq.meter']>468.2274)|(df2['area in sq.meter']<31.40097)] 
##the whiskers values are taken from visualization

In [20]:
outliers_area.shape[0]/df2.shape[0]*100  ##The percentage is to small and can be removed

4.847645429362881

In [21]:
## get index of outliers

In [22]:
idxxx=outliers_area.index

In [23]:
idxxx

Int64Index([  42,   49,   50,   84,   93,  108,  135,  182,  220,  268,  290,
             298,  315,  316,  325,  334,  342,  344,  359,  368,  371,  380,
             435,  442,  464,  470,  497,  525,  535,  574,  604,  613,  626,
             634,  639,  642,  658,  660,  695,  769,  776,  778,  800,  828,
             831,  866,  870,  893,  897,  921,  995,  996, 1001, 1069, 1102,
            1157, 1164, 1193, 1195, 1196, 1201, 1344, 1353, 1362, 1373, 1376,
            1391, 1419, 1424, 1425],
           dtype='int64')

In [24]:
df2.drop(idxxx,inplace=True)



In [25]:
df2.reset_index(inplace=True)

In [26]:
df2

,index,address,price_in_dollars,number_of_bedrooms,number_of_baths,area in sq.meter,city,state
0,0,1866 20th Ave,294900,1.0,1.0,89.557785,Arkdale,Wisconsin
1,1,1639 Pine St,89500,2.0,1.0,85.841695,Arkdale,Wisconsin
2,2,1644 20th Ave,475000,3.0,2.0,202.898551,Arkdale,Wisconsin
3,3,2054 Cumberland Ave,179000,4.0,1.0,162.021553,Arkdale,Wisconsin
4,4,1841 20th Ct,359000,3.0,2.0,156.911929,Arkdale,Wisconsin
...,...,...,...,...,...,...,...,...
1369,1439,"Albany, 12211",735400,2.0,3.0,186.826459,Albany,New york
1370,1440,"Albany, 12211",795400,3.0,3.0,206.800446,Albany,New york
1371,1441,"Albany, 12211",705400,2.0,2.0,183.574879,Albany,New york
1372,1442,"Albany, 12211",815400,4.0,3.0,267.837235,Albany,New york


In [27]:
df2.drop('index',axis=1,inplace=True)

In [28]:
df2

,address,price_in_dollars,number_of_bedrooms,number_of_baths,area in sq.meter,city,state
0,1866 20th Ave,294900,1.0,1.0,89.557785,Arkdale,Wisconsin
1,1639 Pine St,89500,2.0,1.0,85.841695,Arkdale,Wisconsin
2,1644 20th Ave,475000,3.0,2.0,202.898551,Arkdale,Wisconsin
3,2054 Cumberland Ave,179000,4.0,1.0,162.021553,Arkdale,Wisconsin
4,1841 20th Ct,359000,3.0,2.0,156.911929,Arkdale,Wisconsin
...,...,...,...,...,...,...,...
1369,"Albany, 12211",735400,2.0,3.0,186.826459,Albany,New york
1370,"Albany, 12211",795400,3.0,3.0,206.800446,Albany,New york
1371,"Albany, 12211",705400,2.0,2.0,183.574879,Albany,New york
1372,"Albany, 12211",815400,4.0,3.0,267.837235,Albany,New york


## Second step encoding for categorical features


In [29]:
df2['city'].unique()  ##There are 6 unique classes (values), therefore binary encoding can be used for this column

array(['Arkdale', 'Adams', 'Nashville', 'Nolensville', 'Newburgh',
       'Albany'], dtype=object)

In [30]:
df2['state'].unique() ##There are 3 unique classes (values), therefore OneHotEncoder method can be used for this column

array(['Wisconsin', 'Tennessee', 'New york'], dtype=object)

In [31]:
ohe=OneHotEncoder(sparse=False,drop='first')
be=BinaryEncoder()

In [32]:
ohe_df = pd.DataFrame(ohe.fit_transform(df2[['state']]) , columns=ohe.get_feature_names_out())
be_df=be.fit_transform(df2[['city']])


C:\Users\ahmed\anaconda3\envs\newenv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [33]:
ohe_df

,state_Tennessee,state_Wisconsin
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
1369,0.0,0.0
1370,0.0,0.0
1371,0.0,0.0
1372,0.0,0.0


In [34]:
be_df

,city_0,city_1,city_2
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
1369,1,1,0
1370,1,1,0
1371,1,1,0
1372,1,1,0


In [35]:
## now concatenate the encoded features and drop out the main features to be ready for train and test
df2=pd.concat([df2,ohe_df,be_df],axis=1)

In [36]:
df2

,address,price_in_dollars,number_of_bedrooms,number_of_baths,area in sq.meter,city,state,state_Tennessee,state_Wisconsin,city_0,city_1,city_2
0,1866 20th Ave,294900,1.0,1.0,89.557785,Arkdale,Wisconsin,0.0,1.0,0,0,1
1,1639 Pine St,89500,2.0,1.0,85.841695,Arkdale,Wisconsin,0.0,1.0,0,0,1
2,1644 20th Ave,475000,3.0,2.0,202.898551,Arkdale,Wisconsin,0.0,1.0,0,0,1
3,2054 Cumberland Ave,179000,4.0,1.0,162.021553,Arkdale,Wisconsin,0.0,1.0,0,0,1
4,1841 20th Ct,359000,3.0,2.0,156.911929,Arkdale,Wisconsin,0.0,1.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1369,"Albany, 12211",735400,2.0,3.0,186.826459,Albany,New york,0.0,0.0,1,1,0
1370,"Albany, 12211",795400,3.0,3.0,206.800446,Albany,New york,0.0,0.0,1,1,0
1371,"Albany, 12211",705400,2.0,2.0,183.574879,Albany,New york,0.0,0.0,1,1,0
1372,"Albany, 12211",815400,4.0,3.0,267.837235,Albany,New york,0.0,0.0,1,1,0


In [37]:
df2.drop(['city','state'],axis=1,inplace=True)

In [38]:
df2

,address,price_in_dollars,number_of_bedrooms,number_of_baths,area in sq.meter,state_Tennessee,state_Wisconsin,city_0,city_1,city_2
0,1866 20th Ave,294900,1.0,1.0,89.557785,0.0,1.0,0,0,1
1,1639 Pine St,89500,2.0,1.0,85.841695,0.0,1.0,0,0,1
2,1644 20th Ave,475000,3.0,2.0,202.898551,0.0,1.0,0,0,1
3,2054 Cumberland Ave,179000,4.0,1.0,162.021553,0.0,1.0,0,0,1
4,1841 20th Ct,359000,3.0,2.0,156.911929,0.0,1.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1369,"Albany, 12211",735400,2.0,3.0,186.826459,0.0,0.0,1,1,0
1370,"Albany, 12211",795400,3.0,3.0,206.800446,0.0,0.0,1,1,0
1371,"Albany, 12211",705400,2.0,2.0,183.574879,0.0,0.0,1,1,0
1372,"Albany, 12211",815400,4.0,3.0,267.837235,0.0,0.0,1,1,0


## Third step to make train and test and fill the missing values

In [39]:
df2.drop(['address'],axis=1,inplace=True)

In [40]:
x=df2.drop('price_in_dollars',axis=1)

In [41]:
y=df2['area in sq.meter'] ##this is the target

In [42]:
x_train, x_test, y_train,y_test= train_test_split(x , y , test_size=0.2 , random_state=5,shuffle=True )

In [43]:
x_train  ##represents 80 percent of the data

,number_of_bedrooms,number_of_baths,area in sq.meter,state_Tennessee,state_Wisconsin,city_0,city_1,city_2
654,2.0,2.0,105.072464,1.0,0.0,0,1,1
395,4.0,5.0,341.322928,1.0,0.0,0,1,1
1161,2.0,1.0,108.324043,0.0,0.0,1,1,0
1261,6.0,NaN,197.696024,0.0,0.0,1,1,0
1120,5.0,2.0,166.852471,0.0,0.0,1,0,1
...,...,...,...,...,...,...,...,...
73,2.0,2.0,114.548495,1.0,0.0,0,1,1
1142,4.0,3.0,205.314010,0.0,0.0,1,0,1
998,4.0,3.0,199.182460,0.0,0.0,1,0,1
206,3.0,2.0,177.629134,1.0,0.0,0,1,1


In [44]:
y_train.to_frame()  ##represents 20% of the data

,area in sq.meter
654,105.072464
395,341.322928
1161,108.324043
1261,197.696024
1120,166.852471
...,...
73,114.548495
1142,205.314010
998,199.182460
206,177.629134


### Remark: Null values are filled by KNN imputer for number of bedroom, bathroom and area columns as it is the most suitable way as in our case the most close null value to be filled shall be similar to that of the other rows in other features.

In [45]:
knnimp=KNNImputer(n_neighbors=3)

In [46]:
x_train[['number_of_bedrooms','number_of_baths','area in sq.meter']]=knnimp.fit_transform(x_train[['number_of_bedrooms','number_of_baths','area in sq.meter']])


In [47]:
x_train

,number_of_bedrooms,number_of_baths,area in sq.meter,state_Tennessee,state_Wisconsin,city_0,city_1,city_2
654,2.0,2.000000,105.072464,1.0,0.0,0,1,1
395,4.0,5.000000,341.322928,1.0,0.0,0,1,1
1161,2.0,1.000000,108.324043,0.0,0.0,1,1,0
1261,6.0,2.666667,197.696024,0.0,0.0,1,1,0
1120,5.0,2.000000,166.852471,0.0,0.0,1,0,1
...,...,...,...,...,...,...,...,...
73,2.0,2.000000,114.548495,1.0,0.0,0,1,1
1142,4.0,3.000000,205.314010,0.0,0.0,1,0,1
998,4.0,3.000000,199.182460,0.0,0.0,1,0,1
206,3.0,2.000000,177.629134,1.0,0.0,0,1,1


In [48]:
x_test[['number_of_bedrooms','number_of_baths','area in sq.meter']]=knnimp.transform(x_test[['number_of_bedrooms','number_of_baths','area in sq.meter']])
## transform only not fit_transform so that the test train does not get know to any of statistical data x_test

In [49]:
x_test 

,number_of_bedrooms,number_of_baths,area in sq.meter,state_Tennessee,state_Wisconsin,city_0,city_1,city_2
720,2.0,3.0,102.192493,1.0,0.0,0,1,1
1011,3.0,3.0,202.898551,0.0,0.0,1,0,1
1061,3.0,1.0,91.787440,0.0,0.0,1,0,1
51,4.0,4.0,286.231884,1.0,0.0,0,1,1
166,2.0,2.0,106.465998,1.0,0.0,0,1,1
...,...,...,...,...,...,...,...,...
524,4.0,2.0,205.778521,1.0,0.0,0,1,1
352,3.0,4.0,237.458194,1.0,0.0,0,1,1
258,3.0,4.0,171.218878,1.0,0.0,0,1,1
165,1.0,1.0,51.282051,1.0,0.0,0,1,1


## Last step is feature scaling

### Robust scaling is used as there are outliers that are true value

In [50]:
sclr = RobustScaler()

In [51]:
x_train[x_train.columns] = sclr.fit_transform(x_train[x_train.columns])

In [52]:
x_train

,number_of_bedrooms,number_of_baths,area in sq.meter,state_Tennessee,state_Wisconsin,city_0,city_1,city_2
654,-1.0,-1.000000,-0.676067,0.0,0.0,0.0,0.0,0.0
395,1.0,2.000000,1.262195,0.0,0.0,0.0,0.0,0.0
1161,-1.0,-2.000000,-0.649390,-1.0,0.0,1.0,0.0,-1.0
1261,3.0,-0.333333,0.083841,-1.0,0.0,1.0,0.0,-1.0
1120,2.0,-1.000000,-0.169207,-1.0,0.0,1.0,-1.0,0.0
...,...,...,...,...,...,...,...,...
73,-1.0,-1.000000,-0.598323,0.0,0.0,0.0,0.0,0.0
1142,1.0,0.000000,0.146341,-1.0,0.0,1.0,-1.0,0.0
998,1.0,0.000000,0.096037,-1.0,0.0,1.0,-1.0,0.0
206,0.0,-1.000000,-0.080793,0.0,0.0,0.0,0.0,0.0


In [53]:
x_test[x_test.columns] = sclr.transform(x_test[x_test.columns])

In [54]:
x_test

,number_of_bedrooms,number_of_baths,area in sq.meter,state_Tennessee,state_Wisconsin,city_0,city_1,city_2
720,-1.0,0.0,-0.699695,0.0,0.0,0.0,0.0,0.0
1011,0.0,0.0,0.126524,-1.0,0.0,1.0,-1.0,0.0
1061,0.0,-2.0,-0.785061,-1.0,0.0,1.0,-1.0,0.0
51,1.0,1.0,0.810213,0.0,0.0,0.0,0.0,0.0
166,-1.0,-1.0,-0.664634,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
524,1.0,-1.0,0.150152,0.0,0.0,0.0,0.0,0.0
352,0.0,1.0,0.410061,0.0,0.0,0.0,0.0,0.0
258,0.0,1.0,-0.133384,0.0,0.0,0.0,0.0,0.0
165,-2.0,-2.0,-1.117378,0.0,0.0,0.0,0.0,0.0
